# Watermarking experiments (guided)

This notebook helps you run a complete watermark evaluation without needing any technical background.

What you will do:
- Install what is needed.
- Tell the notebook where your images are.
- Choose the watermark model (defaults are sensible).
- Run the evaluation and read the results.

If you are unsure, simply run each cell from top to bottom.

## Step 1 — Set up
We will install the required packages. This may take a minute the first time.

In [ ]:
!pip -q install -r requirements.txt pyyaml


## Step 2 — Point to your images
Please enter the folders containing your images.
- `img_dir` is the folder with images that may contain a watermark.
- `img_dir_nw` is the folder with the matching non‑watermarked images (same filenames).
- `img_dir_fid` is optional and used for the FID score against a reference set. Leave as `None` if unsure.

In [ ]:
img_dir = 'path/to/watermarked_images'
img_dir_nw = 'path/to/non_watermarked_images'
img_dir_fid = None  # e.g. 'path/to/reference_images' or leave as None
output_dir = 'output/notebook_run'


## Step 3 — Choose the watermark model
The default HiDDeN‑style decoder works out of the box. If you downloaded the TorchScript file above, you do not need to change anything below.

If you are using a `.pth` file instead, please set `num_bits`, `redundancy`, `decoder_depth`, and `decoder_channels` accordingly.

In [ ]:
model_name = 'hidden'  # keep as 'hidden' unless you add other models
msg_decoder_path = 'models/dec_48b_whit.torchscript.pt'  # path to your decoder checkpoint
num_bits = 48
redundancy = 1
decoder_depth = 8
decoder_channels = 64
device = 'auto'  # 'auto', 'cpu', or 'cuda'


## Step 4 — Choose what to measure
Leave the defaults if unsure.
- Turn on image quality metrics with `eval_imgs`.
- Turn on bit decoding and robustness with `eval_bits`.
- If you only want the decoded bit‑strings, set `decode_only = True`.
- If decoding with a known key, set `key_str` (a string of 0s and 1s).

In [ ]:
eval_imgs = True
eval_bits = True
decode_only = False
key_str = '111010110101000001010111010011010100010000100111'  # change only if you know the key
batch_size = 32
attack_mode = 'few'  # one of: 'none', 'few', 'all'
save_n_imgs = 5
num_imgs = None  # set to a number to limit how many images are processed


## Step 5 — Create a configuration file (done for you)
We will now write a small configuration file the runner uses.

In [ ]:
import os, yaml
cfg = {
  'output_dir': output_dir,
  'img_dir': img_dir,
  'img_dir_nw': img_dir_nw,
  'img_dir_fid': img_dir_fid,
  'save_n_imgs': save_n_imgs,
  'num_imgs': num_imgs,
  'eval_imgs': eval_imgs,
  'eval_bits': eval_bits,
  'decode_only': decode_only,
  'key_str': key_str,
  'batch_size': batch_size,
  'attack_mode': attack_mode,
  'model': {
    'name': model_name,
    'msg_decoder_path': msg_decoder_path,
    'num_bits': num_bits,
    'redundancy': redundancy,
    'decoder_depth': decoder_depth,
    'decoder_channels': decoder_channels,
    'device': device,
  }
}
os.makedirs('configs', exist_ok=True)
with open('configs/from_notebook.yaml', 'w') as f: yaml.safe_dump(cfg, f)
print('Wrote configs/from_notebook.yaml')


## Step 6 — Run the experiment
This will compute the requested metrics and save results in your `output_dir`.

In [ ]:
!python -m experiments.runner --config configs/from_notebook.yaml


## Step 7 — Read the results
Two CSV files will be produced if the corresponding options were enabled:
- `img_metrics.csv`: image quality metrics between your watermarked and non‑watermarked images.
- `log_stats.csv`: bit accuracy or decoded bit‑strings under various common edits.

The folder `output_dir/imgs/` also contains a few example pairs and their visual differences.

In [ ]:
import os, pandas as pd
from pathlib import Path
print('Output folder:', output_dir)
metrics_csv = Path(output_dir) / 'img_metrics.csv'
stats_csv = Path(output_dir) / 'log_stats.csv'
if metrics_csv.exists():
    display(pd.read_csv(metrics_csv).head())
else:
    print('img_metrics.csv not found (image metrics were likely disabled).')
if stats_csv.exists():
    display(pd.read_csv(stats_csv).head())
else:
    print('log_stats.csv not found (bit evaluation was likely disabled).')
